In [1]:
import requests 
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
import data_utils as du 
from cnn_model import *
from solver import Solver
%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'


In [2]:
X_train, y_train, X_val, y_val, X_test, y_test = du.get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'sgd',
    'lr': 0.01,
    'epoch_num': 20,
}

model_path = 'tmp/trained_20'
full_model = tf.keras.models.load_model(model_path,compile=False)
solver= Solver(full_model,data_set,options=train_options)
test_loss, test_acc = solver.test_model(X_test,y_test)

Loss: 1.2418395280838013,Test Accuracy: 61.76000213623047


## Test the proper quantize Lennet5

In [4]:
import quantize_layers as ql 
import quantize_util as qu 

quantizer = ql.ModelQunatize()
model_option = {
    'conv_reg':0.02, 
}
model_qLennet = ql.proper_quantize_Lennet5(3,10,
                                options=model_option,
                                quantizer = quantizer)
model_qLennet.set_full_precision()
wl = 16 
fl = 8
model_opt ={
    'quantize_wl': wl,
    'quantize_fl': fl,
}

model_q = Quan_Lennet5(3,10,options=model_opt)
## need get one forward the set every thing to train model
#layer_count = model_qLennet.get_quantiable_layer_count()
for var in model_qLennet.trainable_variables:
    print(var.name)
for var in model_q.trainable_variables:
    print(var.name)

quantize_conv2d/kernel:0
quantize_conv2d/bias:0
quantize_conv2d_1/kernel:0
quantize_conv2d_1/bias:0
quantize_dense/kernel:0
quantize_dense/bias:0
quantize_dense_1/kernel:0
quantize_dense_1/bias:0
quantize_dense_2/kernel:0
quantize_dense_2/bias:0
conv2d/kernel:0
conv2d/bias:0
conv2d_1/kernel:0
conv2d_1/bias:0
dense/kernel:0
dense/bias:0
dense_1/kernel:0
dense_1/bias:0
dense_2/kernel:0
dense_2/bias:0


## Test get_quantize_layer function

In [5]:
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
}
model = ql.proper_quantize_Lennet5(3,10,
                                options=model_option,
                                quantizer = quantizer)
solver= Solver(model,data_set,options=train_options,save_model=False)
quantize_layers = model.get_quantizable_layers() 
for layer in quantize_layers:
    print(layer)
layer = quantize_layers[0]
print(layer[0].bias)
model.set_full_precision() 


[<quantize_layers.quantizeConv2D object at 0x7fa0b0206790>, [28, 28, 18]]
[<quantize_layers.quantizeConv2D object at 0x7fa0b03330d0>, [10, 10, 32]]
[<quantize_layers.quantizeDense object at 0x7fa0b02108b0>, [120]]
[<quantize_layers.quantizeDense object at 0x7fa0b0210bb0>, [80]]
[<quantize_layers.quantizeDense object at 0x7fa0b0210eb0>, [10]]
<tf.Variable 'quantize_conv2d_2/bias:0' shape=(18,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)>


## Test model prediciton and Train

In [6]:
import tensorflow_hub as hub
model_path = 'tmp/trained_20'
full_model = hub.KerasLayer(model_path,trainable=True)
qu.copy_weight(model,full_model)
solver.test_model(X_test,y_test)
solver.train()


Loss: 1.2418395280838013,Test Accuracy: 61.76000213623047
Epoch: 0/20
  523/1531 |========*****************|  - train_acc: 0.7087 - train_loss: 0.8455 - val_acc: 0.6340 - val_loss: 1.1969

KeyboardInterrupt: 